## Human-in-the-Loop
There are two main ways to interact with the team from your application:

1. During a team’s run – execution of run() or run_stream(), provide feedback through a UserProxyAgent.

2. Once the run terminates, provide feedback through input to the next call to run() or run_stream().

## Azure Resources Needed
1. Azure OpenAI
    - Deploy GPT-4o


## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## UserProxyAgent - Providing Feedback During a Run
The UserProxyAgent is a special built-in agent that acts as a proxy for a user to provide feedback to the team.

To use the UserProxyAgent, you can create an instance of it and include it in the team before running the team. The team will decide when to call the UserProxyAgent to ask for feedback from the user.

When UserProxyAgent is called during a run, it blocks the execution of the team until the user provides feedback or errors out. This will hold up the team’s progress and put the team in an unstable state that cannot be saved or resumed.

Due to the blocking nature of this approach, it is recommended to use it only for short interactions that require immediate feedback from the user, such as asking for approval or disapproval with a button click, or an alert requiring immediate attention otherwise failing the task.

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback.",
)

# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Whispers of gold in the autumn breeze,
Leaves gently tumbling from tall, ancient trees.
Crisp air whispers secrets, cool on the skin,
As twilight approaches, the night's chill begins.

Pumpkins adorn doorsteps, lanterns aglow,
While squirrels prepare for the coming of snow.
Scarves wrapped snugly, hearts warmed with cheer,
In the embrace of fall, the world feels near.

A palette of crimson, amber, and rust,
Nature's grand finale before winter’s hush.
Footsteps on pathways of leaves gently laid,
Fall's fleeting beauty, destined to fade.
[Prompt tokens: 28, Completion tokens: 119]


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a gentle rhythm. Here are a few points of constructive feedback to consider:

1. **Imagery and Descriptions:**
   - The use of colors like "gold," "crimson," "amber," and "rust" vividly paints the picture of fall.
   - Descriptions like "Leaves gently tumbling from tall, ancient trees" and "Pumpkins adorn doorsteps, lanterns aglow" bring the imagery to life.

2. **Flow and Structure:**
   - The poem has a smooth and consistent flow with a clear structure, making it easy to read.
   - Consider breaking up the poem into stanzas for an even clearer structure.

3. **Themes and Emotions:**
   - The themes of change, preparation for winter, and the fleeting beauty of fall are well-expressed.
   - Phrases like "Crisp air whispers secrets, cool on the skin" and "In the embrace of fall, the world feels near" evoke a warm, nostalgic feeling.

4. **Word Choice:**
   - The language is 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=119), content="Whispers of gold in the autumn breeze,\nLeaves gently tumbling from tall, ancient trees.\nCrisp air whispers secrets, cool on the skin,\nAs twilight approaches, the night's chill begins.\n\nPumpkins adorn doorsteps, lanterns aglow,\nWhile squirrels prepare for the coming of snow.\nScarves wrapped snugly, hearts warmed with cheer,\nIn the embrace of fall, the world feels near.\n\nA palette of crimson, amber, and rust,\nNature's grand finale before winter’s hush.\nFootsteps on pathways of leaves gently laid,\nFall's fleeting beauty, destined to fade.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=372), content='Your poem beautifully captures the essence of the fall season with 

## Using Max Turns

This method allows you to pause the team for user input by setting a maximum number of turns. 

To implement this, set the max_turns parameter in the RoundRobinGroupChat() constructor.

Once the team stops, the turn count will be reset. 

When you resume the team, it will start from 0 again. However, the team’s internal state will be preserved, for example, the RoundRobinGroupChat will resume from the next agent in the list with the same conversation history.

In [ ]:
# Create the team setting a maximum number of turns to 2.
# One chat by agent is one turn. So 2 is needed for the two agents
team = RoundRobinGroupChat([primary_agent, critic_agent], max_turns=2)

task = "Write a short poem about the fall season."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

## HandoffTermination 
HandoffTermination stops the team when an agent sends a HandoffMessage message.

Note: The model used with AssistantAgent must support tool call to use the handoff feature.

In [ ]:
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback first then transfer to user.",
    handoffs=[Handoff(target="user", message="Transfer to user.")],
)

# Define a termination condition that checks for handoff message targetting helper OR text "APPROVE".
handoff_termination = HandoffTermination(target="user")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=handoff_termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)
